In [1]:
# Import libraries
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

In [2]:
data = pd.read_csv('cov_merged.csv')
target = pd.read_csv('outcomes.csv')
print(data)
print(target)

            rowId  covariateId  covariateValue  \
0          3938.0   4083311102               1   
1        175397.0     81380102               1   
2         23031.0   4145418102               1   
3        152472.0    135473102               1   
4         60021.0   4213101102               1   
...           ...          ...             ...   
5411125   67932.0    378253104               1   
5411126  135007.0   4211852104               1   
5411127  127381.0    197988104               1   
5411128   20208.0    201826104               1   
5411129  161935.0    440005104               1   

                                             covariateName  analysisId  \
0        condition_occurrence during day -365 through -...         102   
1        condition_occurrence during day -365 through -...         102   
2        condition_occurrence during day -365 through -...         102   
3        condition_occurrence during day -365 through -...         102   
4        condition_occurrence

In [3]:
# Create windows to represent analysis ids
analysisId_to_window = {101: "all", 102: '365d', 103: '180d', 104: '030d', 105: 'all',
                        106: '365d', 107: '180d', 108: '030d'}

# Create observations data file from data file
observation = pd.DataFrame()
observation['conceptId'] = data['conceptId']
observation['window'] = data['analysisId'].apply(lambda x: analysisId_to_window.get(x, 'Unknown'))
observation['covariateValue'] = 1
observation['patient'] = data['rowId'].astype(int)
print(observation)
print(observation.drop_duplicates())
observation = observation.drop_duplicates()
print(len(set(observation['patient'])))

         conceptId window  covariateValue  patient
0          4083311   365d               1     3938
1            81380   365d               1   175397
2          4145418   365d               1    23031
3           135473   365d               1   152472
4          4213101   365d               1    60021
...            ...    ...             ...      ...
5411125     378253   030d               1    67932
5411126    4211852   030d               1   135007
5411127     197988   030d               1   127381
5411128     201826   030d               1    20208
5411129     440005   030d               1   161935

[5411130 rows x 4 columns]
         conceptId window  covariateValue  patient
0          4083311   365d               1     3938
1            81380   365d               1   175397
2          4145418   365d               1    23031
3           135473   365d               1   152472
4          4213101   365d               1    60021
...            ...    ...             ...      ...
541

In [4]:
max_patient = max(set(observation['patient']))
print(max_patient)
max_concept = max(set(observation['conceptId']))
print(max_concept)

176493
46286052


In [5]:
observation_all = observation[observation['window'] == "all"]
print(len(observation_all))
observation_365d = observation[observation['window'] == "365d"]
print(len(observation_365d))
observation_180d = observation[observation['window'] == "180d"]
print(len(observation_180d))
observation_030d = observation[observation['window'] == "030d"]
print(len(observation_030d))

# Pick time window
chosen_window = "365d"
observation = observation[observation['window'] == chosen_window]

4200550
650535
382161
110648


In [6]:
def pivot_covariates(df):
    observation = df.pivot(index='patient', columns='conceptId', values='covariateValue')
    return observation.fillna(0)

observation_wide = pivot_covariates(observation)

In [7]:
risk_window_end = 1825
def create_outcomes(indexes):
    outcomes = []
    for i in indexes:
        if i in target['rowId'].values:
            days_to_event = target.loc[target['rowId'] == i, 'daysToEvent'].values[0]
            outcomes.append(1 if days_to_event < risk_window_end else 0)
        else:
            outcomes.append(0)
    return outcomes

outcomes = create_outcomes(observation_wide.index)

In [8]:
random_state = 42
test_size = 0.25
# Split the data in train and test set
X_train, X_test, y_train, y_test = train_test_split(observation_wide, outcomes, stratify=outcomes, random_state=random_state, test_size=test_size)

In [9]:
print(X_train)

conceptId  22281     23325     24609     24660     24966     25297     \
patient                                                                 
50529           0.0       0.0       0.0       0.0       0.0       0.0   
19113           0.0       0.0       0.0       0.0       0.0       0.0   
36311           0.0       0.0       0.0       0.0       0.0       0.0   
26132           0.0       0.0       0.0       0.0       0.0       0.0   
163264          0.0       0.0       0.0       0.0       0.0       0.0   
...             ...       ...       ...       ...       ...       ...   
12906           0.0       0.0       0.0       0.0       0.0       0.0   
56633           0.0       0.0       0.0       0.0       0.0       0.0   
50522           0.0       0.0       0.0       0.0       0.0       0.0   
152310          0.0       0.0       0.0       0.0       0.0       0.0   
129118          0.0       0.0       0.0       0.0       0.0       0.0   

conceptId  26727     28060     28396     28457    

In [10]:
df_long = X_train.stack().reset_index()
df_long.columns = ['patient', 'conceptId', 'value']
df_long[df_long['value'] > 1]

,patient,conceptId,value


In [11]:
# Load ancestor and outcomes data
ancestry = pd.read_csv("relations_ancestor_total.csv")
print(ancestry)

          Unnamed: 0  ANCESTOR_CONCEPT_ID  DESCENDANT_CONCEPT_ID  \
0                  1                262.0                  262.0   
1                  2                263.0                  263.0   
2                  3                264.0                  264.0   
3                  4                265.0                  265.0   
4                  5                266.0                  266.0   
...              ...                  ...                    ...   
78195440    78195441           46369493.0             46369493.0   
78195441    78195442           46369844.0             46369844.0   
78195442    78195443           46369845.0             46369845.0   
78195443    78195444           46369901.0             46369901.0   
78195444    78195445           46369902.0             46369902.0   

          MIN_LEVELS_OF_SEPARATION  MAX_LEVELS_OF_SEPARATION         CTID  
0                                0                         0        (0,1)  
1                              

In [12]:
# Choose what level above the data stays
chosen_level = 4
# Filter relations above chosen level
filtered_relations = ancestry[
    (ancestry['ANCESTOR_CONCEPT_ID'] == 441840) &
    (ancestry['MIN_LEVELS_OF_SEPARATION'] < chosen_level)
]

print(filtered_relations)
print(len(set(filtered_relations['DESCENDANT_CONCEPT_ID'])))

        Unnamed: 0  ANCESTOR_CONCEPT_ID  DESCENDANT_CONCEPT_ID  \
517810      517811             441840.0             45765959.0   
517811      517812             441840.0              4151309.0   
517813      517814             441840.0             42535668.0   
517816      517817             441840.0              4095355.0   
517821      517822             441840.0              4036777.0   
...            ...                  ...                    ...   
645041      645042             441840.0              4059280.0   
645046      645047             441840.0              4083769.0   
645047      645048             441840.0              4341796.0   
645050      645051             441840.0              4338249.0   
645051      645052             441840.0              4281541.0   

        MIN_LEVELS_OF_SEPARATION  MAX_LEVELS_OF_SEPARATION        CTID  
517810                         3                         3  (2798,181)  
517811                         2                         2  (

In [27]:
selected_concepts_set = set(filtered_relations['DESCENDANT_CONCEPT_ID'])
print(len(set(selected_concepts_set)))
filtered_cols = selected_concepts_set & set(observation_wide.columns)
print(len(set(filtered_cols)))
print(len(set(observation_wide)))
X_train_filtered = X_train[list(filtered_cols)]
print(X_train_filtered)
X_train_filtered, X_test_filtered = X_train_filtered.align(X_test, join='left', axis=1, fill_value=0)

17829
323
1368
conceptId  4288544  79908    440363   4120624  321588   4030518  440377   \
patient                                                                    
50529          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
19113          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
36311          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
26132          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
163264         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
...            ...      ...      ...      ...      ...      ...      ...   
12906          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
56633          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
50522          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
152310         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
129118         0.0      0.0      0.0      0.0      0.0      0.0      0.0 

In [13]:
# # Select only concepts with observed descendants
# selected_concepts_set = set(filtered_relations['DESCENDANT_CONCEPT_ID'])
# observed_concepts_set = set(X_train.columns)
# observed_descendants = ancestry[ancestry['DESCENDANT_CONCEPT_ID'].isin(observed_concepts_set)]
# concepts_with_observed_descendants_set = set(observed_descendants['ANCESTOR_CONCEPT_ID'])


In [14]:
# # Count how many times each concept appears as an ancestor of observed descendants
# concept_to_score = observed_descendants['ANCESTOR_CONCEPT_ID'].value_counts().to_dict()

In [16]:
import numpy as np
data = pd.DataFrame(np.zeros((115038, 1)), dtype=int)
data.index.name = "patient"
print(data)

         0
patient   
0        0
1        0
2        0
3        0
4        0
...     ..
115033   0
115034   0
115035   0
115036   0
115037   0

[115038 rows x 1 columns]


In [17]:
# propagate observations to selected features
for concept in X_train:
    print(concept)
    selected_concepts = []
    # here, instead of ancestry, we use a simple relationship between concepts for illustration
    observed_descendants = ancestry[ancestry['DESCENDANT_CONCEPT_ID'] == concept]
    concept_ancestors = observed_descendants['ANCESTOR_CONCEPT_ID'].tolist()
    for c in concept_ancestors:
        if c != concept:
            selected_concepts.append(c)
    # selected_concepts = data.columns[(data.columns < concept) & (concept % data.columns == 0)]
    # propagation
    for feature in selected_concepts:
        data[feature] = 0
        data[feature] += X_train[concept] # vectorized propagation
print(data)

22281
23325
24609
24660


C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(ax

24966


C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(ax

25297
26727


...

46270383
46271739


C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(ax

46272749
46273592


C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\1245037232.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0


46286052
         0        432739   432868   434156   439777   440508   441840   \
patient                                                                  
0              0      NaN      NaN      NaN      NaN      NaN      NaN   
1              0      0.0      0.0      0.0      0.0      0.0      0.0   
2              0      0.0      0.0      0.0      0.0      0.0      0.0   
3              0      0.0      0.0      0.0      0.0      0.0      0.0   
4              0      0.0      0.0      0.0      0.0      0.0      0.0   
...          ...      ...      ...      ...      ...      ...      ...   
115033         0      0.0      0.0      0.0      0.0      0.0      0.0   
115034         0      0.0      0.0      0.0      0.0      0.0      0.0   
115035         0      0.0      0.0      0.0      0.0      0.0      0.0   
115036         0      0.0      0.0      0.0      0.0      0.0      0.0   
115037         0      0.0      0.0      0.0      0.0      0.0      0.0   

         443723   443916   4

In [18]:
# from collections import defaultdict

# desc_to_ancestors = defaultdict(set)
# for _, row in ancestry.iterrows():
#     desc_to_ancestors[row['DESCENDANT_CONCEPT_ID']].add(row['ANCESTOR_CONCEPT_ID'])

# for index, row in X_train.iterrows():
#     # Find columns (concepts) with value == 1
#     observed_concepts = row[row == 1].index.astype(int)

#     # Collect all ancestors of the observed concepts
#     ancestors_to_update = set()
#     for concept in observed_concepts:
#         ancestors_to_update.update(desc_to_ancestors.get(concept, set()))

#     # Only update columns that are both ancestors and in X_train.columns
#     valid_ancestors = list(set(X_train.columns.astype(int)) & ancestors_to_update)

#     # Increment those columns for the current row
#     X_train.loc[index, valid_ancestors] += 1


In [19]:
for col in data.columns:
    if col not in X_train.columns:
        X_train[col] = data[col]
X_train = X_train.fillna(0)
print(X_train)

C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

...

C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[col] = data[col]
C:\Users\rbanf\AppData\Local\Temp\ipykernel_32204\2313306436.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

conceptId  22281    23325    24609    24660    24966    25297    26727    \
patient                                                                    
50529          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
19113          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
36311          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
26132          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
163264         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
...            ...      ...      ...      ...      ...      ...      ...   
12906          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
56633          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
50522          0.0      0.0      0.0      0.0      0.0      0.0      0.0   
152310         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
129118         0.0      0.0      0.0      0.0      0.0      0.0      0.0   

conceptId  

In [20]:
from collections import defaultdict

desc_to_ancestors = defaultdict(set)
for _, row in ancestry.iterrows():
    desc_to_ancestors[row['DESCENDANT_CONCEPT_ID']].add(row['ANCESTOR_CONCEPT_ID'])

for index, row in X_train.iterrows():
    observed_concepts = row[row == 1].index.astype(int)

    ancestors_to_update = set()
    for concept in observed_concepts:
        ancestors_to_update.update(desc_to_ancestors.get(concept, set()))
        print(ancestors_to_update)

    valid_ancestors = list(set(X_train.columns.astype(int)) & ancestors_to_update)

    X_train.loc[index, valid_ancestors] += 1
    print(X_train.loc[index, valid_ancestors])

KeyboardInterrupt: 

In [ ]:
# # Apply to training set
# for index, row in X_train.iterrows():
#     print(f"Row {index}:")
#     concepts_with_observed_descendant = []
#     for col in X_train.columns:
#         if row[col] == 1:
#             observed_descendants = ancestry[ancestry['DESCENDANT_CONCEPT_ID'] == col]
#             concept_ancestors = observed_descendants['ANCESTOR_CONCEPT_ID'].tolist()
#             for c in concept_ancestors:
#                 if c != col:
#                     concepts_with_observed_descendant.append(c)
#             print(f"  Observed descendants for {col}: {concept_ancestors}")
            
# #             observed_descendants = ancestry[ancestry['DESCENDANT_CONCEPT_ID'].isin([col])]
# #             concept_with_observed_descendant = observed_descendants['ANCESTOR_CONCEPT_ID']
# #             concepts_with_observed_descendant = concept_with_observed_descendant.tolist()
# #             print(concepts_with_observed_descendant)
#     print(concepts_with_observed_descendant)
    
#     # for col in concepts_with_observed_descendant:
#     #     if col not in X_train.columns:
#     #         X_train[col] = 0 
#     #         row[col] = 0
#     #     X_train.loc[index, col] += 1
#     #     print(f"  {col}: {X_train.loc[index, col]}")
        
#     for col in X_train.columns:
#         if col in concepts_with_observed_descendant:
#             X_train.loc[index, col] += 1
#         print(f"  {col}: {X_train.loc[index, col]}")

            
    
# # X_train['covariateValue'] = X_train.columns.apply(compute_cumulative_score)
# print(X_train)
# print(X_train[X_train['covariateValue'] > 0])

In [ ]:
import pandas as pd
from collections import defaultdict

desc_to_ancestors = defaultdict(list)

for _, row in ancestry.iterrows():
    desc = row['DESCENDANT_CONCEPT_ID']
    anc = row['ANCESTOR_CONCEPT_ID']
    if desc != anc:
        desc_to_ancestors[desc].append(anc)

all_ancestor_cols = set()
for ancestors in desc_to_ancestors.values():
    all_ancestor_cols.update(ancestors)

In [ ]:
X_increment = pd.DataFrame(0, index=X_train.index, columns=X_train.columns)

for index, row in X_train.iterrows():
    active_concepts = row[row == 1].index
    for concept in active_concepts:
        for ancestor in desc_to_ancestors.get(concept, []):
            X_increment.at[index, ancestor] += 1
            print(row, concept, ancestor, X_increment.at[index, ancestor])

X_train += X_increment
print(X_train)


In [ ]:
selected_concepts_set = set(filtered_relations['DESCENDANT_CONCEPT_ID'])
filtered_cols = selected_concepts_set & set(X_train.columns)
X_train_filtered = X_train[list(filtered_cols)]
print(X_train_filtered)
X_train_filtered, X_test_filtered = X_train_filtered.align(X_test, join='left', axis=1, fill_value=0)

In [ ]:
risk_window_end = 1825
def create_outcomes(indexes):
    outcomes = []
    for i in indexes:
        if i in target['rowId'].values:
            days_to_event = target.loc[target['rowId'] == i, 'daysToEvent'].values[0]
            outcomes.append(1 if days_to_event < risk_window_end else 0)
        else:
            outcomes.append(0)
    return outcomes

y_train_filtered = create_outcomes(X_train_filtered.index)

In [ ]:
lr_penalty = 'l1'
lr_solver = "liblinear"
model = LogisticRegression(penalty=lr_penalty, solver=lr_solver)

In [ ]:
# Fit model on train data
model.fit(X_train_filtered, y_train_filtered)


# Get model coefficients
model_coefficients = model.coef_[0]
print(model.coef_)

number_model_features = len(set(model_coefficients))

print('Number of model features:', number_model_features)

# Get train probabilities
y_prob_train = model.predict_proba(X_train_filtered)

# Get ths train auc score
train_auc = roc_auc_score(y_train, y_prob_train[:, 1])
print(train_auc)

# Get test probabilities
y_prob_test = model.predict_proba(X_test_filtered)

# Get ths test auc score
test_auc = roc_auc_score(y_test, y_prob_test[:, 1])
print(test_auc)

# Calculate precision-recall curve
precision_test, recall_test, _ = precision_recall_curve(y_test, y_prob_test[:, 1])

# Calculate AUPRC
auprc_test_score = auc(recall_test, precision_test)